In [1]:
import scvi
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad

/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [68]:
condition_key = 'embryo_id'
label_key = 'cellcluster_moscot'
adata = sc.read_h5ad(f'/lustre/groups/ml01/workspace/monge_velo/data/mouse_gastrulation_atlas/training/adata_500k.h5ad')
obsp = adata.obsp
highly_variable = adata.var_names

In [69]:
adata_copy = adata.copy()
sc.pp.normalize_total(adata_copy)
sc.pp.log1p(adata_copy)
sc.pp.highly_variable_genes(
    adata_copy,
    n_top_genes=2000,
    batch_key=condition_key,
)
highly_variable = adata_copy.var["highly_variable"]
del adata_copy

In [70]:
adata._inplace_subset_var(highly_variable)
var = adata.var

In [21]:
scvi.model.SCVI.setup_anndata(adata, batch_key=condition_key)
vae = scvi.model.SCVI(adata)
vae.train()

/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/jonas.flor/mambaforge/envs/scvi/lib/python ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/jonas.flor/mambaforge/envs/scvi/lib/python ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-7f7d366f-

Epoch 400/400: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [05:14<00:00,  1.46it/s, v_num=1, train_loss_step=860, train_loss_epoch=836]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [05:14<00:00,  1.27it/s, v_num=1, train_loss_step=860, train_loss_epoch=836]


In [22]:
adata_int = ad.AnnData(vae.posterior_predictive_sample().to_scipy_sparse())
adata_int.obs = adata.obs
adata_int.var = adata.var
adata_int.obsm["X_emb"] = vae.get_latent_representation()

In [23]:
adata_int.write('scvi_adata_default_param.h5ad')

In [25]:
np.var(adata_int.obsm["X_emb"], axis=1)

array([0.45612663, 0.5672042 , 1.6021448 , ..., 1.5625286 , 1.0140637 ,
       1.4302133 ], dtype=float32)

# increase number of nodes/layer (double)

In [82]:
scvi.model.SCVI.setup_anndata(adata, batch_key=condition_key)
vae = scvi.model.SCVI(adata, n_hidden=4096, n_latent=50)
vae.train()

/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/jonas.flor/mambaforge/envs/scvi/lib/python ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/jonas.flor/mambaforge/envs/scvi/lib/python ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-7f7d366f-

Epoch 16/16: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [14:32<00:00, 53.61s/it, v_num=1, train_loss_step=603, train_loss_epoch=611]

`Trainer.fit` stopped: `max_epochs=16` reached.


Epoch 16/16: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [14:32<00:00, 54.55s/it, v_num=1, train_loss_step=603, train_loss_epoch=611]


In [83]:
adata_int_2 = ad.AnnData(vae.posterior_predictive_sample().to_scipy_sparse())
adata_int_2.obs = adata.obs
adata_int_2.var = adata.var
adata_int_2.obsm["X_emb"] = vae.get_latent_representation()

In [84]:
np.var(adata_int_2.obsm["X_emb"], axis=1)

array([1.2077645 , 0.5628029 , 1.3952457 , ..., 1.950102  , 1.5745648 ,
       0.71195626], dtype=float32)

In [85]:
adata_int_2.write('scvi_adata_32x_hidden_500k.h5ad')

# increase layers

In [44]:
scvi.model.SCVI.setup_anndata(adata, batch_key=condition_key)
vae = scvi.model.SCVI(adata, n_layers=4, n_latent=50)
vae.train()

/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/jonas.flor/mambaforge/envs/scvi/lib/python ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/jonas.flor/mambaforge/envs/scvi/lib/python ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-7f7d366f-

Epoch 400/400: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [06:38<00:00,  1.21it/s, v_num=1, train_loss_step=770, train_loss_epoch=821]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [06:38<00:00,  1.00it/s, v_num=1, train_loss_step=770, train_loss_epoch=821]


In [45]:
adata_int_3 = ad.AnnData(vae.posterior_predictive_sample().to_scipy_sparse())
adata_int_3.obs = adata.obs
adata_int_3.var = adata.var
adata_int_3.obsm["X_emb"] = vae.get_latent_representation()

In [46]:
np.var(adata_int_3.obsm["X_emb"], axis=1)

array([0.15621844, 0.06291452, 0.1715981 , ..., 0.12979366, 0.20669822,
       0.16187137], dtype=float32)

In [43]:
adata_int_3.write('scvi_adata_2x_layers.h5ad')

# quick transfer

In [2]:
adata_un = sc.read_h5ad('/home/icb/jonas.flor/gastrulation_atlas/scvi/training/500k/all_genes/unintegrated_adata_fixed.h5ad')
adata_un

AnnData object with n_obs × n_vars = 600000 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'batch', 'cell_cluster', 'celltype', 'cellcluster_moscot', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_manager_uuid', '_scvi_uuid'
    obsm: 'X_pca'

In [3]:
vae = scvi.model.SCVI.load(dir_path='/home/icb/jonas.flor/gastrulation_atlas/scvi/training/500k/all_genes', adata=adata_un)

INFO     File /home/icb/jonas.flor/gastrulation_atlas/scvi/training/500k/all_genes/model.pt already downloaded     


/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/jonas.flor/mambaforge/envs/scvi/lib/python ...
  rank_zero_warn(
I0000 00:00:1702374495.171838 3979100 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


In [5]:
adata_int = ad.AnnData(vae.posterior_predictive_sample().to_scipy_sparse())
adata_int.obs = adata_un.obs
adata_int.var = adata_un.var
adata_int.obsm["X_emb"] = vae.get_latent_representation()
adata_int.write('500k_example.h5ad')